# Exploritory Analysis on SHPO OLI Dataset.
By Kellen Bullock

## Framing the Problem and Big Picture
The OLI is a database that contains roughly 65,000 records. Roughly up to 20,000 of these records are dupliates. Several people have been cleaning the data by hand by indicating poss_dup or good in the duplicate_check field. The State Historical Presvation Office then makes the decision to roll those records off the main table into another. 

In order to make more informed deicsions and queries on the database duplicate records should be taken out. Duplicate create confusion for the users by providing redudent information that the user does not want. 


#### Problem Statement
How can automatic complex duplicate record detection be implemented to prevent more duplicate records from entering the dataset?

#### Proposed solution:
Training a neural network to make a propblistic descision on whether a record is a duplicate or not is far more effeicent than checking every record in the dataset. 

Since several thousand records have been classified already that data will be used for supervised learning. 

#### How to solve this problem manually:
A field such as PROPNAME, ADDRESS or RESNAME are sorted alphabetically. First the PROPNAME fields are compared for similarity. Then ADDRESS, RESNAME, ROOF_TYP, and WINDOW_TYPE. If records matchup the records are marked as poss_dup in the duplicate_check column. 

### Peformance measurment:
A CSV file that contains the OBJECTID and probability score that the classifier assigns will be out. This will be joined with the main table dataset and checked by a person. 

Accuracy measrument: RSME
Recall will also be measrued

If the Classifier can out perform at 85% or higher I will call it a success. 

# Assumptions:
<ul>
    <li> The data has complete records, there are no misspelling in the data, There is a combination of numerical, text, and catagorical values. </li>
    <li> The people classifying the records as duplicates are very accurate. 92% accurate or higher.</li>
    <li>

#### Overall project
Title what attempt this is to differintate models.
We will use brnaching in git to help us develop.

#### Toolset
<ul>
    <li>Jupyter notebooks will be for data exploration and visziulation</li>
    <li>Tensorboard will be used for accuracy analysis and cost.</li>
</ul>

#### Notes for Table of Contents:
Establish sections
Enable hyperlinks

#### Put this into the README.md file:
## Objectives:
<ol>
    <li>Name attributes and describe characteristics.</li>
    <ul>
        <li>% of Nulls</li>
        <li>Type of data. ie String, int, float</li>
        <li>Noise present. Such as outliers, logistic, rounding errors</li>
        <li>What is useful what isn't and why</li>
        <li>Type of distribution. </li>
    </ul>
    <li>Identifying Label data</li>
    <li>Visualization of data</li>
    <li>Identify correlations between variables</li>
    <li>Propose how the problem would be solved manually</li>
    <li>Provide transformations if nessiary</li>
    <li>Anything else of interest</li>
</ol>

## Exploritory data Strategy
The nature of the dataset is complex. This is due to the descriptive attirbutes assocatied with properites and cemetaries. There are only a couple of real numerical datatypes such as lat and long. I intend to go through each type figuring out if it is catagorical, a string/text, or numerical. Once the catagroies are identified applying a numerical number scheme for them will be adopted. 

### Questions:
Can I even descrptive statstics on catagorical data? From Comer's class I remember there beng some very strang things that happened.
What do I even do with the catagorical data?

#### Data cleaning Strats:
I could take all null values and fill them with <b>No Data</b>. This would allow for complete records and give vectors to data that isn't there. In turn allowing for the whole record to be proccessed.
<b>I really think this should be prsued, because of the amount of null records there are.</b>
Drawbacks:
    I am creating data and altering data that is in the database. I would have to make general enough preproccessing to catch all of the null or missing values too. Then the model could train but if I miss something it could crash horribily and not really tell me why. Or I could get a very poor level of accuracy. 
    
Implementation:
   <ol>
    <li>Look at all columns and identify incompletes</li>
    <li>Do an fillna() where possible</li>
    <li>Some catagorical data has 00 or none and that would need to change</li>
    <li>Run info() again to see what pandas says.</li>
    <li>Visiually inspect data</li>
   </ol>


# Models
<ol>
    <li>Turn everything into text and concatenate all attributes into one string, apply TF-IDF and cosine similarity, then run model.</li>
    <ul>
        <li><b>Notes</b></li>
        <ul>
            <li>I do not know how and if I need to do describptive stats on the vectors of the strings</li>
        </ul>
    </ul>
    <li>Exploritiory driven modeling</li>
    <ul>
        <li>Convert individual columns into vectors</li>
        <li>PCA (principle compoent analysis vectors and drop relivant columns</li>
        <li>Train model</li>
    </ul>
    <li>Find complete records.</li>
        <ul>
            <li>After discovering complete fields do prepoccessing and run model just on those.</li>
            <li>This strategy will probably not work on the whole dataset becuase there are so many null.</li>
        </ul>
    <li><b>Implement One Hot Encoder instead of TD-IDF.</b></li>
    <ul>
        <li>Problems I forsee with doing this method are there are a lot of nominal values that are misspelled. They create a whole new catagory which will add more complexity to the model. It might not generalize well to new data. For example if we OneHotEncode Oklahoma and Olahoma (county) we create two different catagories. (Contrasting with the TD-IDF method): We are creating vectors based on occurance of important words within the corpus. Okay it seems it may have the same effect.</li>
        <li>This continues to prove that a massive data cleaning proccess needs to be done on the dataset or I need to cherry pick good records out of the dataset.</li>
    </ul>
    

#### Importing modules

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns
import sys

%matplotlib inline
plt.rcParams["figure.figsize"] = [10,10]
pd.set_option('display.max_columns', 70)
pd.set_option('display.max_rows', 200)

#### Loading dataset in:

In [2]:
df = pd.read_excel('datasets/prepared_data/Oklahoma_Working.xls')
df.head()

,PROPNAME,RESNAME,ADDRESS,Lat,Long,duplicate_check
1,STEPHENSON PARK,BRIDGE,,35.65041,-97.47953,0
2,STEPHENSON PARK,PICNIC AREA,,35.65041,-97.47953,0
3,STEPHENSON PARK,CROQUET COURT,,35.65041,-97.47953,0
4,STEPHENSON PARK,TENNIS COURTS,,35.65041,-97.47953,0
5,STEPHENSON PARK,CHILDRENS PLAY AREA,,35.65041,-97.47953,0


# What is each Field?

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3304 entries, 1 to 3948
Data columns (total 6 columns):
PROPNAME           3304 non-null object
RESNAME            3304 non-null object
ADDRESS            3304 non-null object
Lat                3304 non-null float64
Long               3304 non-null float64
duplicate_check    3304 non-null int64
dtypes: float64(2), int64(1), object(3)
memory usage: 180.7+ KB


# Unique Fields

Below is a custom made function designed to display unique values in the dataset all in one table. 

In [4]:
from uniques import uniques

In [5]:
uniques(df)

,duplicate_check
0,0
1,1


In [9]:
df.PROPNAME.unique()

array([' STEPHENSON PARK', ' 1829 MANSFIELD AVE',
       'OKLAHOMA CITY INTENSIVE LEVEL SURVEY PHASE 3',
       '210 EAST 8TH STREET', 'CAMERON BUILDING',
       'LOIS VANWINKLE & JUDY HOKETT', '744 E. ROSE DR. ',
       'CENTRAL BARN SURVEY ', 'STANFORD FURNITURE BUILDING ',
       'OKC DOWNTOWN SURVEY - PHASE 2', 'FILM EXCHANGE HISTORIC DISTRICT',
       'AUTOMOBILE ALLEY HISTORIC DISTRICT', 'SKIRVIN HOTEL',
       "SPEARMAN'S BUILDING",
       'OKLAHOMA CITY INTENSIVE LEVEL SURVEY PHASE 4',
       'ANDERSON ARTIC ICE COMPANY', 'No Data', 'MIDEKE SUPPLY BUILDING ',
       'SWARTZ DEPARTMENT STORE', 'HERITAGE HILLS EAST SURVEY',
       'HOPE COMMUNITY SERVICES INC.', '100 WEST 6TH STREET',
       'CAR AND TRAILOR SHEDS AT ST. HEALTH DEPT.',
       'HANKINSON TIRE COMPANY', '1000 SOUTHWEST 2ND STREET',
       'CENTRAL PARK HISTORIC DISTRICT', 'ZION LUTHERAN CHURCH',
       'CLASSEN TEN PENN HISTORIC DISTRICT', '1004 JASMINE LANE',
       'GEORGE CARTWRIGHT', '1004 NORTHEAST 28TH STREET

The code above does exactly what the table does. As you can see there are multiple errors in the spelling. With no standard of naming was used.

In [10]:
# Sort alphabetical

In [11]:
len(df.PROPNAME.unique().tolist())

667